# Basic tests


In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
pd.DataFrame([
    [1,1]
]).sort_values(by=0)

,0,1
0,1,1


In [ ]:
# test assignment
test_session(
    """
        new Relation(int, int)
        x = 1
        y = 2
        z = y
        Relation($x, $y)
        Relation($y, $x)
        ?Relation(X, $x)
    """,
    pd.DataFrame([[2]],columns=['X']),
)

'?Relation(X,$x)'

,X
1,2


<spannerlib.session.Session>

In [ ]:
# test copy table rule
test_session(
    """
        new B(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        A(X, Y) <- B(X, Y)
        ?A(X, Y)
    """,
    pd.DataFrame(
        [[1,1],[1,2],[2,3]],
        columns=['X','Y']
    )
)

'?A(X,Y)'

,X,Y
0,2,3
1,1,1
2,1,2


<spannerlib.session.Session>

In [ ]:
# test join 2 tables
test_session(
    """
        new B(int, int)
        new C(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        C(2, 2)
        C(1, 1)
        D(X, Y, Z) <- B(X, Y), C(Y, Z)
        ?D(X, Y, Z)
    """,
    pd.DataFrame(
        [[1,2,2],[1,1,1]],
        columns=['X','Y','Z']
    )
)

'?D(X,Y,Z)'

,X,Y,Z
0,1,2,2
1,1,1,1


<spannerlib.session.Session>

In [ ]:
# rel with same free var
test_session(
    """
        new B(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 2)
        A(X) <- B(X, X)
        ?A(X)
    """,
    pd.DataFrame(
        [[1],[2]],
        columns=['X']
    )
)

'?A(X)'

,X
0,1
1,2


<spannerlib.session.Session>

In [ ]:
# union with same vars
test_session(
    """
        new B(int, int)
        new C(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        C(2, 2)
        C(1, 1)

        A(X, Y) <- B(X, Y)
        A(X, Y) <- C(X, Y)
        ?A(X, Y)
    """,
    pd.DataFrame(
        [[1,1],[1,2],[2,2],[2,3]],
        columns=['X','Y']
    )
)

'?A(X,Y)'

,X,Y
0,2,3
1,1,2
2,2,2
3,1,1


<spannerlib.session.Session>

In [ ]:
# union with different vars
test_session(
    """
        new B(int, int)
        new C(int, int)
        B(1, 1)
        B(1, 2)
        B(2, 3)
        C(2, 2)
        C(1, 1)

        A(X, Y) <- B(X, Y)
        A(Z, W) <- C(Z, W)
        ?A(X, Y)
    """,
    pd.DataFrame(
        [[1,1],[1,2],[2,2],[2,3]],
        columns=['X','Y']
    )
)

'?A(X,Y)'

,X,Y
0,2,3
1,1,2
2,2,2
3,1,1


<spannerlib.session.Session>

In [ ]:
# test project
test_session(
    """
            new B(int, int)
            B(1, 1)
            B(1, 2)

            A(X) <- B(X, Y)
            ?A(X)
    """,
    pd.DataFrame(
        [[1]],
        columns=['X']
    )
)

'?A(X)'

,X
0,1


<spannerlib.session.Session>

In [ ]:
# add fact after rule
test_session(
    """
            new B(int, int)
            B(1, 1)
            A(X, Y) <- B(X, Y)
            B(1, 2)
            ?A(Z, W)
    """,
    pd.DataFrame(
        [[1,1],[1,2]],
        columns=['Z','W']
    )
)

'?A(Z,W)'

,Z,W
0,1,1
1,1,2


<spannerlib.session.Session>

In [ ]:
# test data types
test_session(
    """
            new B(int, str, float,bool)
            B(1, "2", -3.5,False)
            ?B(X, Y, Z,W)
    """,
    pd.DataFrame(
        [[1,"2",-3.5,False]],
        columns=['X','Y','Z','W']
    )
)

'?B(X,Y,Z,W)'

,X,Y,Z,W
0,1,2,-3.5,False


<spannerlib.session.Session>

In [ ]:
# join same relation
test_session(
    """
            new Parent(str, str)
            Parent("Sam", "Noah")
            Parent("Noah", "Austin")
            Parent("Austin", "Stephen")


            GrandParent(G, C) <- Parent(G, M), Parent(M, C)
            ?GrandParent(X, "Austin")
    """,
    pd.DataFrame(
        [['Sam']],
        columns=['X']
    )
)

'?GrandParent(X,"Austin")'

,X
1,Sam


<spannerlib.session.Session>

In [ ]:
# test_rule_with_constant
test_session(
"""
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)
    A(X) <- B(1, X)
    ?A(X)
""",
pd.DataFrame([[1],[2]],columns=['X'])
)

'?A(X)'

,X
0,1
1,2


<spannerlib.session.Session>

In [ ]:
# test_rule_with_true_value
test_session(
[
    """
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)

    A(X, Y) <- B(X, Y), C(1, 1)
    """,
    """
    ?A(X, Y)
    """
],
[
    None,   
    pd.DataFrame([[1,1],[1,2],[2,3]],columns=['X','Y'])
],debug=True
)


    ?A(X, Y)
    


(   X  Y
 0  2  3
 1  1  1
 2  1  2,
 defaultdict(<function spannerlib.engine.compute_node.<locals>.<lambda>()>,
             {'C': [None,
                  0  1
               0  1  1
               1  2  2],
              2: [None,
                  0  1
               0  1  1],
              'B': [None,
                  0  1
               0  2  3
               1  1  1
               2  1  2],
              3: [None,
                  _F0  _F1
               0    1    1],
              0: [None,
                  X  Y
               0  2  3
               1  1  1
               2  1  2],
              4: [None,
               Empty DataFrame
               Columns: []
               Index: [0]],
              1: [None,
                  X  Y
               0  2  3
               1  1  1
               2  1  2],
              5: [None,
                  X  Y
               0  2  3
               1  1  1
               2  1  2],
              6: [None,
                  X  Y
       

In [ ]:
# test_rule_with_false_value
test_session(
"""
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)

    A(X, Y) <- B(X, Y), C(0, 0)
    ?A(X, Y)
""",
pd.DataFrame([],columns=['X','Y'])
)

'?A(X,Y)'

,X,Y


<spannerlib.session.Session>

In [ ]:
# test_query_with_same_var
test_session(
[
    """
    new B(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)

    A(X, Y) <- B(X, Y)
    ?A(X,Y)
    """,
    """
    ?A(X, X)
"""
],[
    pd.DataFrame([[1,1],[1,2],[2,3]],columns=['X','Y']),
    pd.DataFrame([[1]],columns=['X']),
],
)

'?A(X,Y)'

,X,Y
0,2,3
1,1,1
2,1,2


'?A(X,X)'

,X
1,1


<spannerlib.session.Session>

In [ ]:
# test_rule_with_constant
test_session(
[
    """
    new B(int, int)
    new C(int, int)
    B(1, 1)
    B(1, 2)
    B(2, 3)
    C(2, 2)
    C(1, 1)

    A(X) <- B(1, X)
    ?A(X)
"""
],[
    pd.DataFrame([[1],[2]],columns=['X']),
],
)

'?A(X)'

,X
0,1
1,2


<spannerlib.session.Session>

In [ ]:
# test_select_and_join
test_session(
[
    """
            new B(int)
            new C(int, int)
            B(2)
            C(1, 4)
            C(2, 5)
            A(X) <- B(X), C(X, 5)
            ?A(X)
"""
],[
    pd.DataFrame([[2]],columns=['X']),
],
)

'?A(X)'

,X
0,2


<spannerlib.session.Session>

In [ ]:
# test Boolean values
test_session(
[
    """
            new A(int)
            A(1)
            ?A(1)
""",
"""?A(2)"""
],[
    True,
    False
],
)

'?A(1)'

True

'?A(2)'

False

<spannerlib.session.Session>

In [ ]:
# test no code doesnt break
test_session(
[
    """""",
],[
    None,
],
)

<spannerlib.session.Session>

In [ ]:
# test_add and remove fact
test_session(
[
    """
    new rel(int)
    +rel(8)
    rel(16)
    -rel(16)
    rel(32)
    rel(16)
    -rel(32)
    ?rel(X)
"""
],[
    pd.DataFrame([[8],[16]],columns=['X']),
],
)

'?rel(X)'

,X
0,8
2,16


<spannerlib.session.Session>

In [ ]:
# combine constants in head with regular heads
test_session(
[
    """
    new BadPeople(str,str)
    BadPeople("John","bad")
    
    new GoodPeople(str)
    GoodPeople("Bob")
    
    PeopleAffinity(Name,Aff)<-BadPeople(Name,Aff)
    PeopleAffinity(Name,"good")<-GoodPeople(Name)
    ?PeopleAffinity(Name,Aff)
"""
],[
    pd.DataFrame([
        ['John','bad'],
        ['Bob','good']
    ],columns=['Name', 'Aff']),
],
)

'?PeopleAffinity(Name,Aff)'

,Name,Aff
0,John,bad
1,Bob,good


<spannerlib.session.Session>